This script performs initial preprocessing of the LMS data:
- Merge individual school-level files
- Join them with the VCCS admin data
- Drop the course sections which show up in Canvas data but not in admin data
- Create the current and historical LMS data respectively

In [6]:
import pandas as pd
import numpy as np
from collections import Counter
import glob

In [7]:
fname_list = [e for e in glob.glob("~\\Box Sync\\Clickstream\\data\\LMS_measures_*.csv") if "20210921" not in e]

In [8]:
crosswalk = pd.read_csv("~/Downloads/UVA_2_UCI_2_CANVAS.CSV", low_memory=False).drop(['research_id', 'canvas_id_pd'], axis=1)
crosswalk = crosswalk[~pd.isnull(crosswalk.canvas_id)]
crosswalk.shape

(546255, 2)

In [9]:
sn_dict = {"BRCC": "Blue Ridge",
           "CVCC": "Central Virginia",
           "DSLCC":  "Dabney S. Lancaster",
           "DCC": "Danville",
           "ESCC": "Eastern Shore",
           "GCC": "Germanna",
           "JSRCC": 'J. Sargeant Reynolds',
           "JTCC": 'John Tyler',
           "LFCC": "Lord Fairfax",
           "MECC": "Mountain Empire",
           "NRCC": "New River",
           "NVCC": "Northern Virginia",
           "PHCC": "Patrick Henry",
           "PDCCC": "Paul D. Camp",
           "PVCC": "Piedmont Virginia",
           "RCC": "Rappahannock",
           "SSVCC": "Southside Virginia",
           "SWVCC": "Southwest Virginia",
           "TNCC": "Thomas Nelson",
           "TCC": "Tidewater",
           "VHCC": "Virginia Highlands",
           "VWCC": "Virginia Western",
           "WCC": "Wytheville"}

In [245]:
vccs_admin = pd.read_stata("~\\Desktop\\2019_2021_vccs_courses.dta")
vccs_admin.loc[:,'admin_available'] = 1

In [249]:
canvas_n = 0
admin_n = 0
check_df_1_list = []
check_df_2_list = []
merged_historical_list = []
merged_final_list = []
for fn in fname_list:
    sn = fn.split("_")[2].split(".")[0]
    
    pvcc_click = pd.read_csv(fn)
    pvcc_click = pvcc_click[~pd.isnull(pvcc_click.sis_course_section_code)]
    pvcc_click.loc[:,'sis_course_section_code'] = pvcc_click.sis_course_section_code.apply(lambda x: x.split("-")[0])
    pvcc_click.loc[:,'sis_course_code'] = pvcc_click.apply(lambda x: "" if x.sis_course_code == x.sis_course_section_code else x.sis_course_code, axis=1)
    sn_code = pvcc_click.sis_course_code.iloc[0].split(".")[0]
    pvcc_click.loc[:,'term'] = pvcc_click.sis_course_section_code.apply(lambda x: x.split(".")[-1])
    pvcc_click = pvcc_click[np.array(pvcc_click.term == "FA19") | np.array(pvcc_click.term == "SU19") | \
                            np.array(pvcc_click.term == "SU20") | \
                            np.array(pvcc_click.term == "SP21") | np.array(pvcc_click.term == "FA20")]

    pvcc_click.loc[:,'section'] = pvcc_click.sis_course_section_code.apply(lambda x: x.split(".")[-2])
    pvcc_click.loc[:,'section_2'] = pvcc_click.sis_course_code.apply(lambda x: x.split(".")[-2] if x != "" and pd.isnull(x) == False else "")
    pvcc_click.loc[:,'subject'] = pvcc_click.sis_course_section_code.apply(lambda x: x.split(".")[-4])
    pvcc_click.loc[:,'course_num'] = pvcc_click.sis_course_section_code.apply(lambda x: x.split(".")[-3])
    # pvcc_click = pvcc_click[pvcc_click.course_num.apply(lambda x: len(x) <= 3)]
    pvcc_click.loc[:,'course'] = pvcc_click.subject + "_" + pvcc_click.course_num
    pvcc_click = pvcc_click.drop(['sis_course_code', 'subject', 'course_num'], axis=1)
    term_dict = {'FA19': 2194, "SU19": 2193, "SU20": 2203, 'FA20': 2204, 'SP21': 2212}
    pvcc_click.loc[:,'strm'] = pvcc_click.term.apply(lambda x: term_dict[x])
    pvcc_click = pvcc_click.drop(['term'], axis=1)
    pvcc_click = pvcc_click.rename(columns = {'canvas_user_id':'canvas_id'})

    pvcc_canvas_id = pvcc_click.loc[:,['canvas_id']].drop_duplicates().sort_values(['canvas_id'])
    print("Total # of unique {} Canvas IDs:".format(sn), len(np.unique(pvcc_canvas_id.canvas_id)))

    pvcc_canvas_id = pvcc_canvas_id.merge(crosswalk, on=['canvas_id'], how='left')
    pvcc_canvas_id.to_csv("~\\Box Sync\\Clickstream\\data\\full\\canvas_id\\{}_CANVAS_ID.csv".format(sn), index=False)
    print("Share of {} Canvas IDs that cannot link to vccsid:".format(sn), np.mean(pd.isnull(pvcc_canvas_id.vccsid)))
    
    pvcc_click = pvcc_click.merge(crosswalk, how='left', on=['canvas_id'])
    pvcc_click_part1 = pvcc_click[~pd.isnull(pvcc_click.vccsid)]
    pvcc_click_part2 = pvcc_click[pd.isnull(pvcc_click.vccsid)]
    print(pvcc_click_part2.shape[0], pvcc_click_part1.shape[0], pvcc_click_part2.shape[0]/pvcc_click_part1.shape[0])
    
    # All available course sections during 2193 - 2212
    all_available_canvas = pvcc_click_part1.loc[:,['strm', 'course', 'section']].drop_duplicates()
    all_available_canvas_2 = pvcc_click_part1.loc[:,['strm', 'course', 'section_2']].drop_duplicates().rename(columns={'section_2':'section'})
    all_available_canvas = pd.concat([all_available_canvas, all_available_canvas_2]).drop_duplicates()
    all_available_canvas.loc[:,'canvas_available'] = 1
    
    pvcc_admin = vccs_admin[vccs_admin.college == sn_dict[sn]].copy()
    pvcc_admin = pvcc_admin.drop(['college'], axis=1)
    print(pvcc_admin.shape) # original # of obs in pvcc admin data
    unavailable_canvas = pvcc_admin.merge(all_available_canvas, on=['strm', 'course', 'section'], how='left')
    unavailable_canvas = unavailable_canvas[pd.isnull(unavailable_canvas.canvas_available)].loc[:,['strm', 'course', 'section']].drop_duplicates()
    pvcc_admin = all_available_canvas.loc[:,['strm', 'course', 'section']].merge(pvcc_admin, on=['strm', 'course', 'section'], how='left')
    print(pvcc_admin.shape) # num of obs in pvcc admin data after excluding those course x section not available in Canvas
    print(np.mean(pd.isnull(pvcc_admin.admin_available))) # Among the linked available course x section obs, share of obs which didn't show up in pvcc admin
    pvcc_admin = pvcc_admin[~pd.isnull(pvcc_admin.admin_available)]
    print(pvcc_admin.shape)
    print(unavailable_canvas.shape[0], all_available_canvas.shape[0])
    
    pvcc_click_part1_new = pvcc_admin.loc[:,['strm', 'course', 'section']].drop_duplicates().merge(pvcc_click_part1,
                                                                                                   on=['strm', 'course', 'section'], how='inner')
    pvcc_click_part1_new_additional = pvcc_admin.loc[:,['strm', 'course', 'section']].drop_duplicates().rename(columns = {'section':'section_2'}).merge(pvcc_click_part1, on=['strm', 'course', 'section_2'], how='inner')
    pvcc_click_part1_new = pd.concat([pvcc_click_part1_new, pvcc_click_part1_new_additional]).drop_duplicates()
    print(pvcc_click_part1.shape[0], pvcc_click_part1_new.shape[0])
    assert pd.isnull(pvcc_click_part1_new.vccsid).any() == False
    assert pd.isnull(pvcc_admin.vccsid).any() == False
    
    ### Drop strm x course x section that has no student activity:
    zero_click_section_1 = pvcc_click_part1_new.loc[:,['strm','course','section','tot_click_cnt']].fillna(0).groupby(['strm','course','section']).agg('sum').reset_index()
    temp = pvcc_click_part1_new.loc[:,['strm','course','section','section_2','tot_click_cnt']]
    temp.loc[:,'section'] = temp.apply(lambda x: x.section_2 if x.section_2 != '' else x.section, axis=1)
    zero_click_section_2 = temp.fillna(0).groupby(['strm','course','section']).agg('sum').reset_index()
    nonzero_click_section = pd.concat([zero_click_section_1, zero_click_section_2]).groupby(['strm','course','section']).agg('max').reset_index()
    nonzero_click_section = nonzero_click_section[nonzero_click_section.tot_click_cnt > 0].drop(['tot_click_cnt'], axis=1)
    pvcc_click_part1_new = pd.concat([pvcc_click_part1_new.merge(nonzero_click_section, how='inner', on=['strm','course','section']), 
                                      pvcc_click_part1_new.merge(nonzero_click_section.rename(columns={'section':'section_2'}), how='inner', on = ['strm','course','section_2'])]).drop_duplicates()
    pvcc_admin = pvcc_admin.merge(nonzero_click_section, how='inner', on=['strm','course','section'])
    
    ### share of obs in Canvas but not in PVCC admin
    pvcc_merged_1 = pvcc_click_part1_new.merge(pvcc_admin, on=['strm','course','section','vccsid'], how='left')
    pvcc_merged_1_part1 = pvcc_merged_1[~pd.isnull(pvcc_merged_1.admin_available)]
    pvcc_merged_1_part2 = pvcc_merged_1[pd.isnull(pvcc_merged_1.admin_available)]
    pvcc_merged_1_part2 = pvcc_merged_1_part2.drop(['grade', 'admin_available', 'credit'], axis=1).merge(pvcc_admin.rename(columns={'section': 'section_2'}), on=['strm','course','section_2','vccsid'], how='left')
    pvcc_merged_1_part2.loc[:,'section'] = pvcc_merged_1_part2.apply(lambda x: x.section_2 if x.admin_available==1 else x.section, axis=1)
    # pvcc_merged_1 = pd.concat([pvcc_merged_1_part1, pvcc_merged_1_part2]).drop(['section_2'], axis=1)
    pvcc_merged_1 = pvcc_merged_1.drop_duplicates()
    section_count = pvcc_merged_1.groupby(['strm', 'course', 'section']).agg({'vccsid':'count'}).reset_index().rename(columns={"vccsid":'count'})
    pvcc_merged_1 = pvcc_merged_1.merge(section_count, how='inner', on=['strm', 'course', 'section'])
    pvcc_merged_1 = pvcc_merged_1.sort_values(['strm','course','vccsid','count','section'])
    pvcc_merged_1 = pvcc_merged_1.fillna(0).groupby([e for e in pvcc_merged_1.columns.values[:-3] if e != "section"]).first().reset_index().replace(0,np.nan)
    pvcc_merged_1.loc[:,'credit'] = pvcc_merged_1.credit.apply(lambda x: 0 if pd.isnull(x) else x)
    nonzero_click_section = pvcc_merged_1.loc[:,['strm','course','section','tot_click_cnt']].fillna(0).groupby(['strm','course','section']).agg('sum').reset_index()
    nonzero_click_section = nonzero_click_section[nonzero_click_section.tot_click_cnt != 0].drop(['tot_click_cnt'], axis=1)
    pvcc_merged_1 = pvcc_merged_1.merge(nonzero_click_section, on = ['strm','course','section'], how='inner')
    pvcc_admin = pvcc_admin.merge(nonzero_click_section, on = ['strm','course','section'], how='inner')
    print(sum(pd.isnull(pvcc_merged_1.admin_available)), sum(pd.isnull(pvcc_merged_1.admin_available))/pvcc_merged_1.shape[0])
    print(pvcc_merged_1.tot_session_cnt.fillna(0).describe())
    print(pvcc_merged_1[pvcc_merged_1.grade == "W"].tot_session_cnt.fillna(0).describe())
    check_df_1 = pvcc_merged_1[np.array(pd.isnull(pvcc_merged_1.admin_available))]
    print(check_df_1.tot_session_cnt.fillna(0).describe())
    print(np.mean(pd.isnull(check_df_1.tot_click_cnt)))
    canvas_n +=pvcc_merged_1.shape[0]
    check_df_1.loc[:,'subject'] = check_df_1.course.apply(lambda x: x.split("_")[0])
    check_df_1.loc[:,'course_num'] = check_df_1.course.apply(lambda x: x.split("_")[1])
    check_df_1 = check_df_1.drop(['admin_available'], axis=1)
    check_df_1.loc[:,'college'] = sn_dict[sn]
    check_df_1 = check_df_1.loc[:,['college', 'strm', 'subject', 'course_num', 'section', 'vccsid'] +\
                               list(np.setdiff1d(list(check_df_1.columns.values),\
                                                 ['college', 'strm', 'subject', 'course_num', 'section', 'vccsid']))]
    check_df_1 = check_df_1.loc[:,['strm','vccsid','course','section','section_2']].merge(pvcc_click, how='inner', on=['strm','vccsid','course','section','section_2']).drop(['strm','vccsid','course','section','section_2'], axis=1)
    check_df_1_list.append(check_df_1.copy())
    assert pd.isnull(pvcc_click_part1_new.canvas_id).any() == False
    print(pvcc_admin.shape[0], pvcc_click_part1_new.shape[0])

    pvcc_merged_2 = pvcc_admin.merge(pvcc_click_part1_new, on=['strm','course','section','vccsid'], how='left')
    print(sum(pd.isnull(pvcc_merged_2.canvas_id)), np.mean(pd.isnull(pvcc_merged_2.canvas_id)))
    check_df_2 = pvcc_merged_2[pd.isnull(pvcc_merged_2.canvas_id)]
    grade_cnt = Counter(check_df_2.grade)
    print(grade_cnt['A']+grade_cnt['B']+grade_cnt['C']+grade_cnt['D'],
          (grade_cnt['A']+grade_cnt['B']+grade_cnt['C']+grade_cnt['D'])/check_df_2.shape[0])
    check_df_2.loc[:,'subject'] = check_df_2.course.apply(lambda x: x.split("_")[0])
    check_df_2.loc[:,'course_num'] = check_df_2.course.apply(lambda x: x.split("_")[1])
    check_df_2 = check_df_2.drop(['course','admin_available'], axis=1)
    check_df_2.loc[:,'college'] = sn_dict[sn]
    check_df_2 = check_df_2.loc[:,['college', 'strm', 'subject', 'course_num', 'section', 'vccsid']]
    check_df_2 = check_df_2.merge(crosswalk, on=['vccsid'], how='left')
    print(np.mean(pd.isnull(check_df_2.canvas_id)))
    # check_df_2 = check_df_2[~pd.isnull(check_df_2.canvas_id)]
    print(check_df_2.shape[0]/pvcc_merged_2.shape[0])
    admin_n += pvcc_merged_2.shape[0]
    check_df_2_list.append(check_df_2.copy())

    pvcc_merged_final = pvcc_click_part1_new.merge(pvcc_admin, on=['strm','course','section','vccsid'], how='inner')
    print(pvcc_merged_final.shape[0], pvcc_click_part1_new.shape[0], pvcc_admin.shape[0])
    print(np.mean(pd.isnull(pvcc_merged_final.tot_click_cnt)))
    pvcc_merged_final.loc[:,'college'] = sn_dict[sn]

    full_term_predictors = [e for e in pvcc_merged_final.columns.values if e.endswith("_qtr1") == False and e.endswith("_qrt1") == False]
    non_predictors = {'strm', 'college', 'course', 'section', 'section_2', 'canvas_course_id',
                      'canvas_course_section_id','canvas_id', 'enrollment_id', 'enrollment_state', 
                      'vccsid', 'credit', 'grade', 'admin_available', 'sis_course_section_code'}
    full_term_predictors = [e for e in full_term_predictors if e not in non_predictors]
    print(len(full_term_predictors))
    qrt1_predictors = [e for e in pvcc_merged_final.columns.values if e.endswith("_qtr1") or e.endswith("_qrt1")]
    print(len(qrt1_predictors))

    pvcc_merged_final.loc[:,'grade'] = pvcc_merged_final.grade.apply(lambda x: np.nan if x == "" else x)
    grade_mode = pvcc_merged_final.groupby(['strm', 'course', 'vccsid']).agg({'grade':pd.Series.mode}).reset_index().rename(columns={'grade':'grade_mode'})
    grade_count = pvcc_merged_final.groupby(['strm', 'course', 'vccsid']).agg({'grade':'count'}).reset_index().rename(columns={'grade':'grade_count'})
    pvcc_merged_final = pvcc_merged_final.merge(grade_mode, on=['strm', 'course', 'vccsid'], how='inner').merge(grade_count, on=['strm', 'course', 'vccsid'], how='inner')
    pvcc_merged_final.loc[:,'grade'] = pvcc_merged_final.apply(lambda x: x.grade_mode if pd.isnull(x.grade) and x.grade_count == 1 else x.grade, axis=1)
    pvcc_merged_final = pvcc_merged_final.drop(['grade_count', 'grade_mode'], axis=1)
    pvcc_merged_final.loc[:,'grade'] = pvcc_merged_final.grade.apply(lambda x: "" if pd.isnull(x) else x)

    pvcc_merged_historical = pvcc_merged_final.loc[:,['strm', 'college', 'course', 'section', 'vccsid', 'grade']+full_term_predictors]
    pvcc_merged_historical = pvcc_merged_historical.sort_values(['strm', 'college', 'course', 'section', 'vccsid'])
    print(pvcc_merged_historical.shape[0])
    merged_historical_list.append(pvcc_merged_historical.copy())

    pvcc_merged_final_new = pvcc_merged_final.loc[:,['strm', 'college', 'course', 'section', 'vccsid', 'grade']+qrt1_predictors]
    print(pd.isnull(pvcc_merged_final_new[pd.isnull(pvcc_merged_final_new.tot_click_cnt_qrt1)]).all())
    merged_final_list.append(pvcc_merged_final_new.copy())
    for cn in qrt1_predictors:
        print(cn, np.mean(pd.isnull(pvcc_merged_final_new[cn])))

    print("\n\n")

Total # of unique BRCC Canvas IDs: 8396
Share of BRCC Canvas IDs that cannot link to vccsid: 0.03870890900428776
764 47555 0.016065608243086952
(39293, 7)
(39472, 7)
0.026753141467369273
(38416, 7)
327 3330


C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



47555 47177
7556 0.2035944278285237
count    37113.000000
mean       130.892598
std        305.967649
min          0.000000
25%         14.000000
50%         75.000000
75%        164.000000
max      30970.000000
Name: tot_session_cnt, dtype: float64
count    1636.000000
mean       64.834352
std        94.140737
min         0.000000
25%        19.000000
50%        40.000000
75%        74.250000
max      1208.000000
Name: tot_session_cnt, dtype: float64
count    7556.000000
mean       13.951826
std       113.003556
min         0.000000
25%         0.000000
50%         0.000000
75%         7.000000
max      8116.000000
Name: tot_session_cnt, dtype: float64
0.5226310217046056


C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


29600 37113
43 0.0014527027027027026
37 0.8604651162790697
1.0
0.0014527027027027026
29557 37113 29600
0.031092465405826032
15
12
29557
strm                       False
college                    False
course                     False
section                    False
vccsid                     False
grade                      False
assign_sub_cnt_qtr1        False
avg_depth_post_qtr1         True
avg_session_len_qrt1        True
avg_word_tot_qtr1           True
disc_post_cnt_qtr1          True
disc_reply_cnt_qtr1         True
disc_tot_messages_qtr1      True
irreg_session_len_qrt1      True
late_assign_cnt_qtr1       False
on_time_assign_cnt_qtr1    False
tot_click_cnt_qrt1          True
tot_time_qrt1               True
dtype: bool
assign_sub_cnt_qtr1 0.561795852082417
avg_depth_post_qtr1 0.5624725107419563
avg_session_len_qrt1 0.04151300876272964
avg_word_tot_qtr1 0.563487498731265
disc_post_cnt_qtr1 0.5691714314713943
disc_reply_cnt_qtr1 0.6839327401292418
disc_tot_messages_qtr1 0.56

6 0.0013366005791935842
5 0.8333333333333334
1.0
0.0013366005791935842
4483 5504 4489
0.015168414008476466
15
12
4483
strm                       False
college                    False
course                     False
section                    False
vccsid                     False
grade                      False
assign_sub_cnt_qtr1        False
avg_depth_post_qtr1        False
avg_session_len_qrt1        True
avg_word_tot_qtr1          False
disc_post_cnt_qtr1         False
disc_reply_cnt_qtr1         True
disc_tot_messages_qtr1     False
irreg_session_len_qrt1      True
late_assign_cnt_qtr1       False
on_time_assign_cnt_qtr1    False
tot_click_cnt_qrt1          True
tot_time_qrt1               True
dtype: bool
assign_sub_cnt_qtr1 0.41155476243586886
avg_depth_post_qtr1 0.6636181128708454
avg_session_len_qrt1 0.03702877537363373
avg_word_tot_qtr1 0.6660718269016284
disc_post_cnt_qtr1 0.6654026321659603
disc_reply_cnt_qtr1 0.7537363372741468
disc_tot_messages_qtr1 0.6636181128708454


47092 58333
59 0.0012528667289560859
29 0.4915254237288136
1.0
0.0012528667289560859
47033 58333 47092
0.029064699253715475
15
12
47033
strm                       False
college                    False
course                     False
section                    False
vccsid                     False
grade                      False
assign_sub_cnt_qtr1        False
avg_depth_post_qtr1         True
avg_session_len_qrt1        True
avg_word_tot_qtr1           True
disc_post_cnt_qtr1          True
disc_reply_cnt_qtr1         True
disc_tot_messages_qtr1      True
irreg_session_len_qrt1      True
late_assign_cnt_qtr1       False
on_time_assign_cnt_qtr1    False
tot_click_cnt_qrt1          True
tot_time_qrt1               True
dtype: bool
assign_sub_cnt_qtr1 0.525694724980333
avg_depth_post_qtr1 0.45429804605277146
avg_session_len_qrt1 0.04084366296004933
avg_word_tot_qtr1 0.45937958454702016
disc_post_cnt_qtr1 0.4660770097591053
disc_reply_cnt_qtr1 0.6024918674122425
disc_tot_messages_qtr1 0

9833 12403
40 0.004067934506254449
31 0.775
0.925
0.004067934506254449
9793 12403 9833
0.03819054426631267
15
12
9793
strm                       False
college                    False
course                     False
section                    False
vccsid                     False
grade                      False
assign_sub_cnt_qtr1        False
avg_depth_post_qtr1        False
avg_session_len_qrt1        True
avg_word_tot_qtr1          False
disc_post_cnt_qtr1         False
disc_reply_cnt_qtr1         True
disc_tot_messages_qtr1     False
irreg_session_len_qrt1      True
late_assign_cnt_qtr1       False
on_time_assign_cnt_qtr1    False
tot_click_cnt_qrt1          True
tot_time_qrt1               True
dtype: bool
assign_sub_cnt_qtr1 0.3170632084141734
avg_depth_post_qtr1 0.4961707341978965
avg_session_len_qrt1 0.050444194833044013
avg_word_tot_qtr1 0.49851935055651997
disc_post_cnt_qtr1 0.501072194424589
disc_reply_cnt_qtr1 0.6822219953027673
disc_tot_messages_qtr1 0.4961707341978965


19035 22432
49 0.0025742054110848437
36 0.7346938775510204
0.8163265306122449
0.0025742054110848437
18986 22432 19035
0.06536395238596861
15
12
18986
strm                       False
college                    False
course                     False
section                    False
vccsid                     False
grade                      False
assign_sub_cnt_qtr1        False
avg_depth_post_qtr1         True
avg_session_len_qrt1        True
avg_word_tot_qtr1           True
disc_post_cnt_qtr1          True
disc_reply_cnt_qtr1         True
disc_tot_messages_qtr1      True
irreg_session_len_qrt1      True
late_assign_cnt_qtr1       False
on_time_assign_cnt_qtr1    False
tot_click_cnt_qrt1          True
tot_time_qrt1               True
dtype: bool
assign_sub_cnt_qtr1 0.4815126935636785
avg_depth_post_qtr1 0.5131675971768672
avg_session_len_qrt1 0.11266196144527546
avg_word_tot_qtr1 0.5212261666491099
disc_post_cnt_qtr1 0.5181186137153693
disc_reply_cnt_qtr1 0.7761508479932582
disc_tot_me

15908 20710
24 0.0015086748805632386
18 0.75
1.0
0.0015086748805632386
15884 20710 15908
0.03191891211281793
15
12
15884
strm                       False
college                    False
course                     False
section                    False
vccsid                     False
grade                      False
assign_sub_cnt_qtr1        False
avg_depth_post_qtr1        False
avg_session_len_qrt1        True
avg_word_tot_qtr1          False
disc_post_cnt_qtr1         False
disc_reply_cnt_qtr1        False
disc_tot_messages_qtr1     False
irreg_session_len_qrt1      True
late_assign_cnt_qtr1       False
on_time_assign_cnt_qtr1    False
tot_click_cnt_qrt1          True
tot_time_qrt1               True
dtype: bool
assign_sub_cnt_qtr1 0.5521908839083355
avg_depth_post_qtr1 0.6553135230420549
avg_session_len_qrt1 0.05854948375723999
avg_word_tot_qtr1 0.6566356081591539
disc_post_cnt_qtr1 0.6607277763787459
disc_reply_cnt_qtr1 0.8173004281037523
disc_tot_messages_qtr1 0.655313523042054

In [250]:
check_df_1_final = pd.concat(check_df_1_list)
check_df_1_final.to_csv("~\\Box Sync\\Clickstream\\data\\full\\not_in_vccs_admin_data.csv", index=False)
print(check_df_1_final.shape[0] / canvas_n)
check_df_2_final = pd.concat(check_df_2_list)
check_df_2_final.sort_values(['strm', 'college', 'subject', 'course_num', 'section', 'vccsid']).to_csv("~\\Box Sync\\Clickstream\\data\\full\\not_in_canvas_data.csv", index=False)
print(1 - check_df_2_final.shape[0]/admin_n)
vccs_merged_historical = pd.concat(merged_historical_list)
vccs_merged_historical.to_stata("~\\Box Sync\\Clickstream\\data\\full\\LMS_data_historical.dta", write_index=False)

0.24099096235489662
0.9984172271370284


In [251]:
vccs_merged_final_new = pd.concat(merged_final_list)
for cn in vccs_merged_final_new.columns.values:
    print(cn, np.mean(pd.isnull(vccs_merged_final_new[cn])))

strm 0.0
college 0.0
course 0.0
section 0.0
vccsid 0.0
grade 0.0
assign_sub_cnt_qtr1 0.49144960409828137
avg_depth_post_qtr1 0.5392928363070696
avg_session_len_qrt1 0.041547028129237794
avg_word_tot_qtr1 0.5422760903382315
disc_post_cnt_qtr1 0.5460139142379187
disc_reply_cnt_qtr1 0.6629204671175811
disc_tot_messages_qtr1 0.5392928363070696
irreg_session_len_qrt1 0.04987090078586434
late_assign_cnt_qtr1 0.8954543827869745
on_time_assign_cnt_qtr1 0.6811458802084166
tot_click_cnt_qrt1 0.041547028129237794
tot_time_qrt1 0.041547028129237794


In [252]:
vccs_merged_final_new = vccs_merged_final_new.sort_values(['strm', 'college', 'course', 'section', 'vccsid'])
print(vccs_merged_final_new.shape[0])
# vccs_merged_final_new = vccs_merged_final_new[vccs_merged_final_new.course.apply(lambda x: len(x) == 7)]
# print(vccs_merged_final_new.shape[0])
vccs_merged_final_new.to_stata("~\\Box Sync\\Clickstream\\data\\full\\updated\\LMS_data.dta", write_index=False)

1313331
